<a href="https://colab.research.google.com/github/camen6ert/word_level_pytorch_lstm/blob/main/jokes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl

#VERSION = "nightly"  #@param ["1.5" , "20200325", "nightly"]
#!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
#!python pytorch-xla-env-setup.py --version $VERSION

In [30]:
# imports pytorch
import torch

!nvidia-smi

# imports the torch_xla package
#import torch_xla
#import torch_xla.core.xla_model as xm

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [31]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

train_on_gpu = False

if device == 'cuda':
  train_on_gpu = True

print('Using {} device'.format(device))

#device = xm.xla_device()
#print(device)

Using cpu device


In [32]:
import re

f = open("/content/drive/MyDrive/Colab Notebooks/data/witze2.txt", "r")

jokes = f.read()

jokes = re.sub(r'[^\w\s\n]','',jokes.lower())
jokes = jokes.split("\n")
print(jokes[:50])

['zwei bauern am stammtisch ich hab letzte woche alle meine tiere markieren müssen mit nem ring im linken ohr sauarbeit sag ich dir kann ich mir denken die ganzen kühe schweine schafe ja aber das schlimmste waren die bienen ', 'ein scharr nonnen kam zu petrus und wollt in den himmel petrus zur ersten nonne hast du schonmal einen penis angefasst die nonne ja aber nur mit der fingerspitze petrus dann tauche deine fingerspitze ins weihwasser und geh in den himmel petrus zur zweiten nonne hast du schonmal einen penis angefasst die nonne naja ich muss gestehen ich habe ihn massiert petrus dann tauche deine hand ins weihwasser und geh in den himmel plötzlich entsteht unruhe in der schlange weil sich eine der nonnen vorgedrängelt hat petrus fragt die nonne warum drängelst du dich vor die nonne na wenn ich das zeug schon gurgeln muss will ich es tun bevor schwester maria ihren arsch reinhält ', 'die tochter kommt vor der hochzeitsnacht zu ihrer mutter mama ich hab da noch eine frage die mutter

In [33]:
from collections import Counter

words = [word.split(" ") for word in jokes]
all_words = [w for word in words for w in word]

#word_buf = set()
#for word in words:
#  for w in word:
#    word_buf.add(w)
print(all_words[:10])
word_counts = Counter(all_words)
vocab = sorted(word_counts, key=word_counts.get, reverse=True)

word2idx = {word: idx for (idx, word) in enumerate(vocab)}
idx2word = {idx: word for (idx, word) in enumerate(vocab)}

wordsNr = [word2idx[w] for word in words for w in word]

print(word2idx['bauer'])

['zwei', 'bauern', 'am', 'stammtisch', 'ich', 'hab', 'letzte', 'woche', 'alle', 'meine']
299


In [34]:
import numpy as np
from torch.utils.data import TensorDataset, DataLoader

sequence_len = 6


In [35]:
train_ds = []
train_label_ds = []

for i in range(len(wordsNr)//sequence_len):
  k=0
  train = wordsNr[i:i+sequence_len]
  label = wordsNr[i+1:i+1+sequence_len]

  #padding
  if len(label) < sequence_len:
    y = sequence_len - len(label)
    for j in range(sequence_len - len(label)):
      label.append(0)

  train2 = [t for t in train]
  label2 = [l for l in label]

  train_ds.append(train2)
  train_label_ds.append(label2)

train_np = np.array(train_ds)
train_label_np = np.array(train_label_ds)

tensor_x = torch.LongTensor(train_np)
tensor_y = torch.LongTensor(train_label_np)

print(tensor_x.shape)
print(tensor_y.shape)

train_dataset = TensorDataset(tensor_x, tensor_y) # create your datset
dataloader = DataLoader(train_dataset, batch_size=10, drop_last=True) # create your dataloader


torch.Size([71418, 6])
torch.Size([71418, 6])


In [36]:
import torch
from torch import nn

class Model(nn.Module):
    def __init__(self, num_layers):
        super(Model, self).__init__()
        self.lstm_size = 256
        self.embedding_dim = 256
        self.num_layers = num_layers

        n_vocab = len(idx2word)
        self.embedding = nn.Embedding(
            num_embeddings=n_vocab,
            embedding_dim=self.embedding_dim,
        )
        self.lstm = nn.LSTM(
            input_size=self.lstm_size,
            hidden_size=self.lstm_size,
            num_layers=self.num_layers,
            batch_first=True
        )
        self.fc = nn.Linear(self.lstm_size, len(vocab))


    def forward(self, x, prev_state):
 
        embed = self.embedding(x)
        output, state = self.lstm(embed, prev_state)
        #output, state = self.lstm2(embed, state)
        output = self.fc(output)
        return output, state

    def init_hidden(self, bts):
        return (torch.zeros(self.num_layers, bts, self.lstm_size).to(device),
                torch.zeros(self.num_layers, bts, self.lstm_size).to(device))

In [37]:
import argparse
import torch
import numpy as np
from torch import nn, optim
from torch.utils.data import DataLoader


def train(dataset, model, batch_size, epoch, sequence_length):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    model.train()

    for i in range(epoch):
      (h, c) = model.init_hidden(batch_size)
      myLoss = 0
      k = 0

      for idx, (inputs, labels) in enumerate(dataset):

        model.zero_grad()
        inputs, labels = inputs.to(device), labels.to(device)
        
        output, (h, c) = model(inputs, (h, c))
        
        loss = criterion(output.transpose(1, 2), labels)

        h = h.detach()
        c = c.detach()

        loss.backward()
        optimizer.step()

        nn.utils.clip_grad_norm_(model.parameters(), 5)

        myLoss += loss.item()
        k = idx

        #if idx % 1000 == 999:
        # print({ 'epoch': i, 'idx': idx, 'loss': loss.item() })          

      print({ 'epoch': i, 'loss': myLoss/k })
     # torch.save(model.state_dict(), '/content/drive/MyDrive/Colab Notebooks/data/joke_model')


In [38]:
model = Model(5)

model.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/data/joke_model', map_location=torch.device(device)))

model = model.to(device)
print(device)

train(dataloader, model, batch_size=10, epoch=30, sequence_length=sequence_len)

cpu


KeyboardInterrupt: ignored

In [34]:
torch.save(model.state_dict(), '/content/drive/MyDrive/Colab Notebooks/data/joke_model')

In [39]:
newModel = Model(5)

newModel.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/data/joke_model', map_location=torch.device('cpu')))
newModel.eval()

Model(
  (embedding): Embedding(24563, 256)
  (lstm): LSTM(256, 256, num_layers=5, batch_first=True)
  (fc): Linear(in_features=256, out_features=24563, bias=True)
)

In [40]:
newModel.eval()


text = "sagt die nonne zum bauern"
words = text.split(' ')
h = newModel.init_hidden(5)

for i in range(0, 15):
  arr = [word2idx[w] for w in words[i:]]
  x = torch.tensor(arr)

  x = x.unsqueeze(-1)
  x = x.to(device)
  y_pred, h = newModel(x, h)
  last_word_logits = y_pred[0][-1]

  p = torch.nn.functional.softmax(last_word_logits, dim=0).detach().numpy()
  word_index = np.random.choice(len(last_word_logits), p=p)
  words.append(idx2word[word_index])

print(words)
#word_index = np.random.choice(len(last_word_logits), p=p)


['sagt', 'die', 'nonne', 'zum', 'bauern', 'das', 'liebe', 'das', 'letzten', 'holger', 'ist', 'mama', 'ist', 'angefasst', 'na', 'erwiederte', 'zu', 'nicht', 'die', 'sagt']
